### Метрическое Обучение (Metric Learning)

#### Введение в Метрическое Обучение
Метрическое обучение — это способ обучения моделей для измерения сходства или различий между различными объектами. Основная цель — научить модель выделять такие признаки, чтобы объекты одного и того же класса находились близко в пространстве признаков, а разных классов — дальше друг от друга.

<img width=500 src='https://cdn.dida.do/blog/20220926_wc_metric-learning/face_recognition.png' />

#### Дано
- Набор данных: $ X = \{x_1, x_2, ..., x_n\} $
- Метки классов: $ Y = \{y_1, y_2, ..., y_n\} $
- Метрика расстояния: $ D $ (например, Евклидово расстояние)

#### Цель

Обучить нейронную сеть $f$, такую, что:

- $D(f(x_1), f(x_2)) -> min$, если $y_1=y_2$
- $D(f(x_1), f(x_2)) -> max$, если $y_1\neq y_2$

#### Сиамские Сети
Сиамские сети представляют собой две идентичные нейронные сети, обучающиеся параллельно. Они генерируют векторы признаков для лиц, стремясь минимизировать расстояние между векторами одного лица и максимизировать его между разными лицами.

![](https://miro.medium.com/v2/resize:fit:1400/0*lgjFPlTjPjiW4ziu.png)

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*uEf5xti1DhFkJ_jEZKLNgg.png)

#### Функции Потерь в Метрическом Обучении

##### Contrastive Loss
- **Формула**: $ L = Y \frac{1}{2}(D_W)^2 + (1 - Y) \frac{1}{2}(\max(0, m - D_W))^2 $
  - $ D_W $ — расстояние между парами векторов.
  - $ Y $ — метка сходства (1 для похожих, 0 для разных).
  - $ m $ — пороговое значение.
- **Интуиция**: Минимизация расстояния для похожих пар и увеличение для разных.

![](https://miro.medium.com/v2/resize:fit:786/format:webp/1*Du1JOucg-090mSpjRhPxuA.png)


## Triplet

![](https://miro.medium.com/v2/resize:fit:1016/0*5XAJbso7L_3En7cJ.png)


##### Triplet Margin Loss
- **Формула**: $ L = \max(D(A, P) - D(A, N) + margin, 0) $
  - $ A $ — якорь (вектор лица).
  - $ P $ — положительный пример (похожее лицо).
  - $ N $ — отрицательный пример (разное лицо).
- **Интуиция**: Разделение расстояний между якорем и положительными/отрицательными примерами.

![](https://miro.medium.com/v2/resize:fit:1204/1*QDFExMf0FL1ZFYn4PRMyPw.png)

![](https://hav4ik.github.io/articles/images/2021-06-14-deep-metric-learning-survey/metric_losses.png)

[Библиотека метрического обучения](https://github.com/KevinMusgrave/pytorch-metric-learning)


![](https://www.researchgate.net/publication/340541441/figure/fig3/AS:878611699621888@1586488761053/Embedding-of-the-training-and-test-sets-of-MNIST-dataset-in-the-feature-spaces-of.ppm)

#### Функции Потерь на Основе Центров

Центровые функции потерь вносят дополнительные ограничения на обучение, формируя более четкие кластеры внутри классов и улучшая разделимость между разными классами. Они особенно полезны в задачах, где различия между классами нечетки и сложны для разделения.

##### Center Loss
- **Формула**: $ L_c = \frac{1}{2} \sum_{i=1}^m \|x_i - c_{y_i}\|^2 $
  - $ x_i $ — вектор признаков образца.
  - $ c_{y_i} $ — центр класса образца.
- **Интуиция**: Сокращение расстояния между образцами и центром их класса.

![](https://raw.githubusercontent.com/shunk031/chainer-center-loss/master/img/visualize_deep_feature_with_center_loss.png)

![](https://miro.medium.com/v2/resize:fit:2000/1*yXLT1O2fWQyiY9Syy25Xag.png)

### Softmax Loss
Softmax loss обычно выражается формулой:

$$
L = -\frac{1}{N} \sum_{i=1}^{N}\log \left( \frac{e^{W_{y_i}^T z_i + b_{y_i}}}{\sum_{j} e^{W_j^T z_i + b_j}} \right)
$$

где:
- $ W_{y_i} $ — веса классификатора для класса $ y_i $,
- $ z_i $ — вектор признаков для $ i $-го примера,
- $ b_{y_i} $ — смещение (bias) для класса $ y_i $,
- $ e^{W_j^T z_i + b_j} $ — экспоненцированное скалярное произведение вектора признаков и весов классификатора для всех классов $ j $.

### SphereFace Loss (Sphere Loss)

#### Основная Идея
SphereFace Loss обучает модель таким образом, чтобы угловое расстояние между объектами разных классов было максимальным, в то время как угловое расстояние между объектами одного класса минимизируется. Это достигается за счет введения угловой маргинальности в стандартную функцию потерь softmax.

### Модификации для SphereFace Loss
1. **Удаление Bias ($ b $)**:
   Для упрощения и увеличения устойчивости модели к изменениям масштаба признаков bias убирается из уравнения.

2. **Нормализация Весов ($ ||W|| = 1 $)**:
   Веса классификатора $ W $ нормализуются так, чтобы их норма была равна единице. Это помогает модели сосредоточиться на угловом расстоянии между векторами.

3. **Угловая Модификация**:
   Скалярное произведение $ W_{y_i}^T x_i $ заменяется на $ ||W_{y_i}|| \cdot ||x_i|| \cdot \cos(\theta) $, где $ \theta $ — угол между $ x_i $ и $ W_{y_i} $. Поскольку $ ||W_{y_i}|| = 1 $, уравнение упрощается до $ ||x_i|| \cdot \cos(\theta) $.

### Переход к SphereFace Loss
Функция потерь SphereFace может быть выражена следующей формулой:
$$
L = -\frac{1}{N} \sum_{i=1}^{N} \log \frac{e^{\| \mathbf{x}_i \| \cos(m \theta_{y_i, i})}}{e^{\| \mathbf{x}_i \| \cos(m \theta_{y_i, i})} + \sum_{j \neq y_i} e^{\| \mathbf{x}_i \| \cos(\theta_{j, i})}}
$$
где:
- $ N $ — количество объектов в батче.
- $ \mathbf{x}_i $ — вектор признаков $ i $-го объекта.
- $ y_i $ — истинный класс $ i $-го объекта.
- $ \theta_{y_i, i} $ — угол между вектором признаков $ \mathbf{x}_i $ и весом класса $ y_i $ в последнем слое сети.
- $ m $ — маргинальный параметр, который увеличивает угловое расстояние для правильных классов, делая классификацию более жесткой.


### Объяснение Модификаций
- Удаление bias упрощает модель и делает ее более устойчивой к масштабным изменениям признаков.
- Нормализация весов и переход к угловым расстояниям помогают модели фокусироваться на угловом расположении классов на сферической поверхности, что улучшает разделяющую способность модели и делает ее более робастной к изменениям нормы векторов признаков.
- Использование углового расстояния вместо скалярного произведения учитыв

## ArcFace и CosFace

### ArcFace
ArcFace (Additive Angular Margin Loss) добавляет угловую маржу к углу между вектором признаков и соответствующим весом класса в softmax loss. Формула ArcFace выглядит следующим образом:

$$
L_{\text{ArcFace}} = -\frac{1}{N} \log \left( \frac{e^{s \cdot \cos(\theta_{y_i} + m)}}{\sum_{j} e^{s \cdot \cos(\theta_j)}} \right)
$$

где:
- $ \theta_{y_i} $ — угол между вектором признаков $ z_i $ и весом класса $ W_{y_i} $,
- $ s $ — масштабный коэффициент,
- $ m $ — угловая маржа, добавляемая к углу $ \theta_{y_i} $.

### CosFace
CosFace (Large Margin Cosine Loss) модифицирует softmax loss, добавляя фиксированное косинусное расстояние между вектором признаков и весами класса. Формула CosFace:

$$
L_{\text{CosFace}} = -\frac{1}{N} \log \left( \frac{e^{s \cdot (\cos(\theta_{y_i}) - m)}}{\sum_{j} e^{s \cdot \cos(\theta_j)}} \right)
$$

где:
- $ \theta_{y_i} $ — как и в ArcFace, угол между $ z_i $ и $ W_{y_i} $,
- $ s $ — масштабный коэффициент,
- $ m $ — косинусное расстояние, вычитаемое из косинуса угла $ \theta_{y_i} $.

### Общие Характеристики и Различия
И ArcFace, и CosFace вводят дополнительные параметры (угловую маржу в ArcFace и косинусное расстояние в CosFace), которые помогают повысить разделяющую способность модели между разными классами. Эти модификации способствуют обучению более дискриминативных признаков, улучшая качество распознавания лиц. Основное различие между ними заключается в способе применения маржи: ArcFace добавляет угловую маржу непосредственно к углу, в то время как CosFace вычитает фиксированное значение из косинуса угла.

![](https://laniakea2018.files.wordpress.com/2018/06/l09_decisionmarginwithdifferentlossfnt.jpg)